In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data,housing.target,random_state=42
)

X_train,X_valid,y_train,y_valid = train_test_split(
    X_train_full,y_train_full,random_state=42
)

In [3]:
mlp_reg = MLPRegressor(hidden_layer_sizes=[50,50,50],random_state=42)
pipeline = make_pipeline(StandardScaler(),mlp_reg)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=[50, 50, 50],
                              random_state=42))])

In [4]:
y_pred = pipeline.predict(X_valid)

In [5]:
rmse = mean_squared_error(y_valid,y_pred,squared=False)

e:\ML & Data\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [6]:
rmse

0.5053326657968658

**Teraz za pomocą interfejsu Keras sekwencyjnego**

In [7]:
import tensorflow as tf

In [8]:
norm_layer = tf.keras.layers.Normalization(input_shape = X_train.shape[1:])

e:\ML & Data\.venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50,activation="relu"),
    tf.keras.layers.Dense(50,activation="relu"),
    tf.keras.layers.Dense(50,activation="relu"),
    tf.keras.layers.Dense(1)
    ])

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mse',optimizer=optimizer,metrics = ['RootMeanSquaredError'])
norm_layer.adapt(X_train)
history = model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 1.4364 - loss: 2.2041 - val_RootMeanSquaredError: 1.7722 - val_loss: 3.1406
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - RootMeanSquaredError: 0.6278 - loss: 0.3942 - val_RootMeanSquaredError: 0.9541 - val_loss: 0.9104
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - RootMeanSquaredError: 0.6005 - loss: 0.3607 - val_RootMeanSquaredError: 1.1592 - val_loss: 1.3437
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - RootMeanSquaredError: 0.5907 - loss: 0.3492 - val_RootMeanSquaredError: 0.8102 - val_loss: 0.6564
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - RootMeanSquaredError: 0.5838 - loss: 0.3416 - val_RootMeanSquaredError: 0.9020 - val_loss: 0.8136
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - RootMeanSquaredError: 0.5730 - loss: 0.3286 - val_RootMeanSquaredError: 0.9517 - val_loss: 0.9058
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - RootMeanSquaredError: 

In [11]:
mse_test, rmse_test = model.evaluate(X_test,y_test)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - RootMeanSquaredError: 0.5255 - loss: 0.2764


In [12]:
X_new = X_test[:3]
y_pred = model.predict(X_new)
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


array([[0.46786135],
       [1.1835127 ],
       [4.8986564 ]], dtype=float32)

**Teraz za pomocą interfejsu keras funkcyjnego**

In [13]:
normalization_layer = tf.keras.layers.Normalization() # standaryzuje dane wejściowe
hidden_layer1 = tf.keras.layers.Dense(30,activation='relu')
hidden_layer2 = tf.keras.layers.Dense(30,activation='relu')
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)
input_layer = tf.keras.layers.Input(shape = X_train.shape[1:])

In [14]:
normalized = normalization_layer(input_layer)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized,hidden2])
output = output_layer(concat)

In [15]:
model = tf.keras.Model(inputs = [input_layer],outputs = [output])

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mse',optimizer=optimizer,metrics = ['RootMeanSquaredError'])
normalization_layer.adapt(X_train)
history = model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 1.8524 - loss: 3.5614 - val_RootMeanSquaredError: 1.5473 - val_loss: 2.3940
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - RootMeanSquaredError: 0.6994 - loss: 0.4895 - val_RootMeanSquaredError: 0.9431 - val_loss: 0.8894
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - RootMeanSquaredError: 0.6334 - loss: 0.4014 - val_RootMeanSquaredError: 1.3699 - val_loss: 1.8765
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - RootMeanSquaredError: 0.6474 - loss: 0.4193 - val_RootMeanSquaredError: 1.1344 - val_loss: 1.2869
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - RootMeanSquaredError: 0.5993 - loss: 0.3594 - val_RootMeanSquaredError: 1.8652 - val_loss: 3.4792
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - RootMeanSquaredError: 0.6120 - loss: 0.3747 - val_RootMeanSquaredError: 2.5303 - val_loss: 6.4026
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - RootMeanSquaredError: 